In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten,Input
from keras.optimizers import Adam
import tensorflow as tf
from keras_tuner import RandomSearch, BayesianOptimization
import keras
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import random

 

# Load the data
data = pd.read_csv('../../../Data/Final_df/BTC_FINAL_df_class.csv')


# MAke the date the index
data.set_index('Date', inplace=True)

seed_value = 123
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

# Normalize the data
# List of columns to scale
columns_to_scale = ['Open', 'High', 'Low','Close', 'Volume', 'RSI', 'ATR', 'MACD', 'MFI',
                    'EMA', 'SMA', 'OBV', 'GTrends_Interest', 'Sentiment_Bullish',
                    'Price_oil', 'Price_gold', 'Price_NASDAQ', 'Price_SP500', 'Price_NYSE',
                    'Interest_Rate', 'hash_rate', 'users']

# Initialize the scaler
scaler = MinMaxScaler(feature_range=(0, 1))

# Scale the selected columns
data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

# Train and test data
training_size = int(len(data) * 0.9)
training_data = data[:training_size]
test_data = data[training_size:]

train_data = training_data[:int(len(training_data) * 0.9)]
val_data = training_data[int(len(training_data) * 0.9):]


# Define the function to create the dataset
def create_dataset(data, window_size, target_index):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data.iloc[i:(i + window_size)].values)  
        y.append(data.iloc[i + window_size, target_index])
    return np.array(X), np.array(y)

# Forecast Horizon
window_size = 3

# Target index ('Close')
close_index = 22

# Create the dataset
X_train, y_train = create_dataset(train_data, window_size, close_index)
X_test, y_test = create_dataset(test_data, window_size, close_index)
X_val, y_val = create_dataset(val_data, window_size, close_index)
X_train_full, y_train_full = create_dataset(training_data, window_size, close_index)


# Reshape the data
X_train = X_train.reshape((X_train.shape[0], window_size, X_train.shape[2], 1))
X_test = X_test.reshape((X_test.shape[0], window_size, X_test.shape[2], 1))
X_val = X_val.reshape((X_val.shape[0], window_size, X_val.shape[2], 1))
X_train_full = X_train_full.reshape((X_train_full.shape[0], window_size, X_train_full.shape[2], 1))


# Define a model-building function
def build_model(hp):
    model = Sequential()
    filters = hp.Int('filters', min_value=10, max_value=150, step=5)
    pool = hp.Choice('pool_size', values=[2, 3])
    model.add(Conv2D(filters=filters,
                    kernel_size=3, activation='relu', padding='same', input_shape=(X_train.shape[1], X_train.shape[2], 1)))
    model.add(MaxPooling2D(pool_size=pool, padding='same'))
    
    model.add(Conv2D(filters=filters, 
                     kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=pool, padding='same'))

    model.add(Conv2D(filters=filters, 
                     kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=pool,padding='same'))

    model.add(Flatten())

    model.add(Dense(1, activation='sigmoid'))  
    
    model.compile(optimizer=Adam(
        learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',  
        metrics=['accuracy'])  
    
    return model

# Try1:CNN-3D-CLASS
# Bets : CNN-3D-CLASS-2
# Best : CNN-3D-CLASS-3

# Instantiate the tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy', 
    max_trials=200,
    executions_per_trial=1,
    directory='my_dir',
    project_name='CNN-3D-CLASS-3',
    overwrite=False
)


# Configure EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_accuracy',   
    patience=15,          
    restore_best_weights=True  
)

# Execute the search with EarlyStopping
tuner.search(
    X_train, y_train,  
    epochs=100,
    batch_size=32,
    validation_data=(X_val, y_val),  
    verbose=2,
    callbacks=[early_stopping]
)

# Get the best model
best_hps = tuner.get_best_hyperparameters()[0]
best_model = tuner.hypermodel.build(best_hps)

print(best_model.summary())

Reloading Tuner from my_dir/CNN-3D-CLASS-3/tuner0.json


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 3, 23, 120)     │         1,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 1, 8, 120)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 1, 8, 120)      │       129,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 1, 3, 120)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 1, 3, 120)      │       129,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 1, 1, 120)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 120)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           121 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 260,761 (1018.60 KB)

 Trainable params: 260,761 (1018.60 KB)

 Non-trainable params: 0 (0.00 B)

None


In [10]:
# Fit the model n times and averages the metrics
n = 30
accuracy_list = []
f1_list = []
precision_list = []
recall_list = []

for _ in range(n):
    best_hps = tuner.get_best_hyperparameters()[0]
    best_model = tuner.hypermodel.build(best_hps)
    print("iteration: ", _)
    history = best_model.fit(
        X_train_full, y_train_full,
        epochs=250,
        batch_size=32,
        verbose=0
    )
    predictions = best_model.predict(X_test)
    test_pred = (predictions > 0.5).astype(int)

    accuracy = accuracy_score(y_test, test_pred)
    f1 = f1_score(y_test, test_pred)
    precision = precision_score(y_test, test_pred)
    recall = recall_score(y_test, test_pred)

    accuracy_list.append(accuracy)
    f1_list.append(f1)
    precision_list.append(precision)
    recall_list.append(recall)

# Calculate the average metrics
average_accuracy = np.mean(accuracy_list)
average_f1 = np.mean(f1_list)
average_precision = np.mean(precision_list)
average_recall = np.mean(recall_list)
    
# Print the average metrics
print('Accuracy:', average_accuracy)
print('F1:', average_f1)
print('Precision:', average_precision)
print('Recall:', average_recall)

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


iteration:  0
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
iteration:  1


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
iteration:  2


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
iteration:  28


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
iteration:  29


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Accuracy: 0.47884057971014493
F1: 0.38499868999622905
Precision: 0.47952017570407374
Recall: 0.3373333333333333


In [8]:
# Seed 
seed_value = 42
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

# Train the final model 
final_model = tuner.hypermodel.build(best_hps)
history = final_model.fit(
    X_train_full, y_train_full,
    epochs=100,
    batch_size=32,
    verbose=2
)

# Evaluate the final model on the test data
predictions = final_model.predict(X_test)
test_pred = (predictions > 0.5).astype(int)

final_accuracy = accuracy_score(y_test, test_pred)
final_f1 = f1_score(y_test, test_pred)
final_precision = precision_score(y_test, test_pred)
final_recall = recall_score(y_test, test_pred)

# Print the final metrics
print('Final Model Metrics:')
print('Accuracy:', final_accuracy)
print('F1:', final_f1)
print('Precision:', final_precision)
print('Recall:', final_recall)

# Save the model
#final_model.save('best_CNN3DCLASS.keras')


Epoch 1/100


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


98/98 - 1s - 10ms/step - accuracy: 0.5198 - loss: 0.6929
Epoch 2/100
98/98 - 0s - 4ms/step - accuracy: 0.5285 - loss: 0.6911
Epoch 3/100
98/98 - 0s - 4ms/step - accuracy: 0.5233 - loss: 0.6897
Epoch 4/100
98/98 - 0s - 4ms/step - accuracy: 0.5217 - loss: 0.6887
Epoch 5/100
98/98 - 0s - 4ms/step - accuracy: 0.5240 - loss: 0.6883
Epoch 6/100
98/98 - 0s - 4ms/step - accuracy: 0.5233 - loss: 0.6879
Epoch 7/100
98/98 - 0s - 4ms/step - accuracy: 0.5269 - loss: 0.6871
Epoch 8/100
98/98 - 0s - 4ms/step - accuracy: 0.5275 - loss: 0.6868
Epoch 9/100
98/98 - 0s - 4ms/step - accuracy: 0.5342 - loss: 0.6862
Epoch 10/100
98/98 - 0s - 4ms/step - accuracy: 0.5355 - loss: 0.6858
Epoch 11/100
98/98 - 0s - 4ms/step - accuracy: 0.5342 - loss: 0.6858
Epoch 12/100
98/98 - 0s - 3ms/step - accuracy: 0.5486 - loss: 0.6852
Epoch 13/100
98/98 - 0s - 4ms/step - accuracy: 0.5489 - loss: 0.6843
Epoch 14/100
98/98 - 0s - 4ms/step - accuracy: 0.5460 - loss: 0.6844
Epoch 15/100
98/98 - 0s - 4ms/step - accuracy: 0.5531 